In [ ]:
import pypsa
import pandas as pd
import matplotlib.pyplot as plt
import sys
from pathlib import Path
import os
#os.chdir(os.path.dirname(os.path.abspath(__file__)))
sys.path.append('../scripts/')
sys.path.append('scripts/')
from helpers import load_config

In [ ]:
# # Experiments with mock snakemake
# notebook_dir = Path.cwd()

# if "snakemake" not in globals():
#     from helpers import mock_snakemake

#     os.chdir(os.path.dirname(os.path.abspath(notebook_dir)))
#     snakemake = mock_snakemake("plot_export")

In [ ]:
PREFIX = "/mnt/c/Users/scl38887/Documents/git/pypsa-nz/"
OUTPUT = PREFIX + "results/figures/modelenergy/"
tech_colors = PREFIX + "config/tech_colors.yaml"

In [ ]:
if not os.path.exists(OUTPUT):
    os.makedirs(OUTPUT)

In [ ]:
n_eu = pypsa.Network(PREFIX + "results/modelenergy/ammonia-de/6627cbeac8b58c2e9a5abd9bb456ab1b.nc")

In [ ]:
n_jp = pypsa.Network(PREFIX + "results/modelenergy/ammonia-jp/149f5f53c8d1afe16494dc95b6a8021c.nc")

In [ ]:
n_us = pypsa.Network(PREFIX + "results/modelenergy/ammonia-us/9e668c99305a4e1511d752d73db77fbb.nc")

## Preliminary graphs

### Cost per unit of energy

In [ ]:
cost_us = n_us.statistics.revenue() / n_us.statistics.dispatch().loc[:,"efuel"].values[0] * -1
cost_jp = n_jp.statistics.revenue() / n_jp.statistics.dispatch().loc[:,"efuel"].values[0] * -1
cost_eu = n_eu.statistics.revenue() / n_eu.statistics.dispatch().loc[:,"efuel"].values[0] * -1

In [ ]:
cost_all = pd.concat([cost_us, cost_jp, cost_eu], axis=1, keys=["US", "JP", "EU"])
cost_all = cost_all.reset_index(level='component').drop(columns='component')

In [ ]:
cost_all

In [ ]:
fig, ax = plt.subplots(figsize=(6, 4))

# Get colors from yaml defined in tech_colors
colors = load_config(tech_colors)["me_colors"]

cost_all[cost_all>0].T.plot.bar(ax=ax, stacked=True, alpha=0.8, color=colors)


#plt.xlabel("H2 rule")
plt.ylabel("Costs in €/MWh")
plt.title(f"Cost of delivered ammonia from NZ")
handles, labels = ax.get_legend_handles_labels()
ax.legend(reversed(handles), reversed(labels), loc='upper left', bbox_to_anchor=(1, 1))
plt.xticks(rotation=0)

plt.grid(axis='y', alpha=0.5) 

# fig.savefig(OUTPUT+f"export_comp.pdf", bbox_inches="tight")
fig.savefig(snakemake.output.modelenergy, bbox_inches="tight")